# Read Image data into memory

In [1]:
import time
import getpass
username = getpass.getuser()
start_time = time.time()
#Where is the data?
if username == "bdutt":
    baseDir="/Users/bdutt/Documents/workspace-whitespace/face/face/data/lfw"
elif username =="cgarg":
    baseDir="/Users/cgarg/Documents/charu/intuit/code/face/data/lfw"
flow = "main"
debug = False

%run "Load Data.ipynb"

filenames = []
labels = []

def collectFileNamesAndLabels(filename):
    filenames.append(filename)
    result = filename.split("/") 
    labels.append(result[len(result)-2]) #Get the label name from filename
    

doForEachFile(collectFileNamesAndLabels,baseDir, 1000)
print("Total File name collection time = %s seconds" % (time.time() - start_time))
print ("Files collected : ",len(filenames))

Total File name collection time = 2.4576408863067627 seconds
Files collected :  647


In [2]:
start_time = time.time()
imagesList = []
doForEachFileNames(readImage, filenames, imagesList)
print ("Images data collected : ",len(imagesList))
print("Total data collection time = %s seconds" % (time.time() - start_time))

Images data collected :  647
Total data collection time = 1.7276649475097656 seconds


# Generic Multi threading code

Used later

In [3]:
import threading
import sys
import traceback
import time

class myThread (threading.Thread):
    def __init__(self, threadName, index, worker):
        threading.Thread.__init__(self)
        self.threadName = threadName
        self.index = index
        self.worker = worker
    def run(self):
        print('^',self.threadName, ', ', end='')
        try:
            self.worker(self.threadName, self.index)
        except:
            e = sys.exc_info()[0]
            print("Exception in thread: ",self.threadName,", ", e)
            traceback.print_exc()
        print('V',self.threadName, end='')
        

def runThreads(nThreads, worker):
    threads = []
    for i in range(nThreads):
        try:
            t = myThread("Thread-"+str(i), i , worker)
            threads.append(t)
            t.start()
        except:
            e = sys.exc_info()[0]
            print("Error: unable to start thread: ", e)
            traceback.print_exc()

    time.sleep(1)

    for t in threads:
        t.join()

In [4]:
start_time = time.time()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])


Y = pd.DataFrame(labels, columns = ['name'])
sizeOfYLabel = len(Y)
oneDImageList = []
showOneImage = False

imageListLock = threading.Lock()

def flattenImage(image, imageList):
    # Convert RGB to single number
    global showOneImage
    if showOneImage:
        plt.imshow(image, cmap="Greys")
        plt.show()
    gray = rgb2gray(image)
    if showOneImage:
        plt.imshow(gray, cmap="Greys")
        plt.show()
        showOneImage = False
    imageList.append(gray.flatten())

nThreads = 10
splitImagesList = np.array_split(imagesList, nThreads)
splitLabelsList = np.array_split(labels, nThreads)
labels = []

def imageFlatteningWorker(threadName, index):
    global splitImagesList
    global imageListLock
    imageList = []
    for image in splitImagesList[index]:
        flattenImage(image, imageList)
    imageListLock.acquire()
    global oneDImageList
    oneDImageList.extend(imageList)
    global labels
    labels.extend(splitLabelsList[index])
    imageListLock.release()

runThreads(nThreads, imageFlatteningWorker)
    
del imagesList

X = pd.DataFrame(oneDImageList)
del oneDImageList
print("X shape", X.shape)
print("label count", len(labels))
print("Total dataframe creation time = %s seconds" % (time.time() - start_time))

^ Thread-0 , ^ Thread-1 , ^ Thread-2 , ^ Thread-3 , ^^ Thread-5 ,  Thread-4 , ^ Thread-6 , ^^ Thread-7 ,  Thread-8 , ^ Thread-9 , V Thread-1V Thread-6V Thread-4V Thread-3V Thread-0V Thread-2V Thread-7VV Thread-5 Thread-8V Thread-9X shape (647, 62500)
label count 647
Total dataframe creation time = 11.268031120300293 seconds


In [5]:
#Y['name'].value_counts()

In [6]:
start_time = time.time()
maxFrequency = 21

# We upsample/downsample to have equal quantity of all classes

#X_sample = pd.DataFrame(columns = X.columns)
#Y_sample = pd.DataFrame(columns = Y.columns)
X_sampleList = []
Y_sampleList = []

XY_sampleLock = threading.Lock()

def equalSampleLabel(label):
    global X_sampleList
    global Y_sampleList
    global XY_sampleLock
    dfa = X[Y['name']== label]
    dfasample = dfa.sample(n=maxFrequency,replace=True)
    dfb = Y[Y['name']== label]
    dfbsample = dfb.sample(n=maxFrequency,replace=True)
    print('.', end='')
    XY_sampleLock.acquire()
    print('-', end='') 
    X_sampleList.append(dfasample)
    Y_sampleList.append(dfbsample)
    XY_sampleLock.release()
    print('+', end='')



labels = Y['name'].unique()

nThreads = 10
labelsArray = np.array_split(labels, nThreads)

def worker(threadName, index):
    for label in labelsArray[index]:
        #print(threadName, label)
        equalSampleLabel(label)

runThreads(nThreads, worker)

X_sample = pd.concat(X_sampleList)
Y_sample = pd.concat(Y_sampleList)
print(X_sample.shape)
print(Y_sample.shape)
X = X_sample
Y = Y_sample
# print(X_sample)
print("Total equal sampling time = %s seconds" % (time.time() - start_time))

^ Thread-0 , ^ Thread-1 , ^ Thread-2^  Thread-3^ , ,  Thread-4 , ^ Thread-5 , .-+^ Thread-6 , .^ Thread-7 , -+^ Thread-8 , ^ Thread-9 , .-+..-+.-.+-+...-+-+.-..+-.+-+.-+.-.+.-+-+-+-+.-+..-+-+-..+-.+-+.-+-.+-+.-+.-+.-+.-+..-+..-+-..+-+.-+-+-+.-+.-+.-+.-+..-+-+.-+..-+..-....+.-+-+.-+-+-+-.+-+-+-+..-+-+.-+.-+..-.+-+..-+.-+-+.-+.-+.-+-.+-+..-+.-+-+.-+..-+..-+.-+-+.-+-+..-+-+.-+.-+.-+....-+-+-+-.+.-...+-...+.-+-+-+-+-+-+-+.-+.-+..-+-+.-+.-+.-+...-+-.+-+.-+-+..-+-+.-+.-+....-+.-+-+.-+.-+-+-+.-..+.-+-....+-+..-+-.+-+-..+-+.-.+-+-.+-+-+-..+-+.-+-.+-+...-+-+-.+..-.+-+..-+-+-+-+-+...-+.-.+.-.+-+.-+-+.-+-+.-.+-....+-+.-+.-.+-+-.+-.+-+-+-...+-.+-+-+-...+.-+..-+-+.-+.-+-+-+-+-+.-+..-+-+.-+.-+.-+.-+..-+-+.-+..-+-+.-+..-+.-+-+.-+.-+.-+.-+.-+.-+.-+.-+.-+.-+.-+.-+.-+.-+.-+..-+.-+-+..-+.-+-.+-+.-+.-+.-+..-.+.-.+.-+.-+-..+.-+-+.-+-.+-+-.+.-+.-+.-+-+..-...+-+-+..-+-+.-+-+-+-+.-+.-+.-+.-+.-+.-...+-+..-+-+-+-+..-.+..-+-+..-+-.+-+..-+-.+-+-..+.-+-.+-.+-.+-+.-+-+.-+.-+.-+-+.-+..-+.-+..-.+-+-+.-+-+.-+.-+.-+.-+

In [10]:
start_time = time.time()
sizeOfX = X_sample.shape[1]
%run "DNN.ipynb"

print("Total DNN time = %s seconds" % (time.time() - start_time))

Shape of X to DNN (7455, 62500)
Shape of Y to DNN (7455, 1)
Shape of Y post OneHot encoding (7455, 355)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 50)                3125050   
_________________________________________________________________
dense_5 (Dense)              (None, 25)                1275      
_________________________________________________________________
dropout_2 (Dropout)          (None, 25)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 355)               9230      
Total params: 3,135,555
Trainable params: 3,135,555
Non-trainable params: 0
_________________________________________________________________
Epoch 1/250
7455/7455 [==============================] - 22s 3ms/step - loss: 0.6866 - categorical_accuracy: 0.0036
Epoch 2/250
7455/7455 [=====================

7455/7455 [==============================] - 20s 3ms/step - loss: 0.0242 - categorical_accuracy: 0.0085
Epoch 63/250
7455/7455 [==============================] - 20s 3ms/step - loss: 0.0240 - categorical_accuracy: 0.0086
Epoch 64/250
7455/7455 [==============================] - 20s 3ms/step - loss: 0.0237 - categorical_accuracy: 0.0098
Epoch 65/250
7455/7455 [==============================] - 16s 2ms/step - loss: 0.0234 - categorical_accuracy: 0.0082
Epoch 66/250
7455/7455 [==============================] - 16s 2ms/step - loss: 0.0231 - categorical_accuracy: 0.0098
Epoch 67/250
7455/7455 [==============================] - 16s 2ms/step - loss: 0.0229 - categorical_accuracy: 0.0127
Epoch 68/250
7455/7455 [==============================] - 17s 2ms/step - loss: 0.0228 - categorical_accuracy: 0.0107
Epoch 69/250
7455/7455 [==============================] - 17s 2ms/step - loss: 0.0224 - categorical_accuracy: 0.0146
Epoch 70/250
7455/7455 [==============================] - 16s 2ms/step - loss

7455/7455 [==============================] - 22s 3ms/step - loss: 0.0169 - categorical_accuracy: 0.1622
Epoch 132/250
7455/7455 [==============================] - 22s 3ms/step - loss: 0.0168 - categorical_accuracy: 0.1753
Epoch 133/250
7455/7455 [==============================] - 22s 3ms/step - loss: 0.0168 - categorical_accuracy: 0.1804
Epoch 134/250
7455/7455 [==============================] - 22s 3ms/step - loss: 0.0168 - categorical_accuracy: 0.1763
Epoch 135/250
7455/7455 [==============================] - 23s 3ms/step - loss: 0.0167 - categorical_accuracy: 0.1779
Epoch 136/250
7455/7455 [==============================] - 21s 3ms/step - loss: 0.0165 - categorical_accuracy: 0.1874
Epoch 137/250
7455/7455 [==============================] - 23s 3ms/step - loss: 0.0167 - categorical_accuracy: 0.1765
Epoch 138/250
7455/7455 [==============================] - 23s 3ms/step - loss: 0.0166 - categorical_accuracy: 0.1903
Epoch 139/250
7455/7455 [==============================] - 23s 3ms/ste

7455/7455 [==============================] - 21s 3ms/step - loss: 0.0130 - categorical_accuracy: 0.4079
Epoch 201/250
7455/7455 [==============================] - 21s 3ms/step - loss: 0.0130 - categorical_accuracy: 0.4023
Epoch 202/250
7455/7455 [==============================] - 20s 3ms/step - loss: 0.0130 - categorical_accuracy: 0.4118
Epoch 203/250
7455/7455 [==============================] - 21s 3ms/step - loss: 0.0129 - categorical_accuracy: 0.4141
Epoch 204/250
7455/7455 [==============================] - 21s 3ms/step - loss: 0.0129 - categorical_accuracy: 0.4138
Epoch 205/250
7455/7455 [==============================] - 21s 3ms/step - loss: 0.0128 - categorical_accuracy: 0.4224
Epoch 206/250
7455/7455 [==============================] - 21s 3ms/step - loss: 0.0127 - categorical_accuracy: 0.4186
Epoch 207/250
7455/7455 [==============================] - 21s 3ms/step - loss: 0.0127 - categorical_accuracy: 0.4239
Epoch 208/250
7455/7455 [==============================] - 21s 3ms/ste

In [11]:
# print(Y)
enc.get_feature_names(['name'])
print(enc.get_feature_names(['name']).shape)

(355,)


In [20]:
import numpy as np
import tensorflow as tf

Ypred = pd.DataFrame( columns = ['sampleId', "classAsIndex", "classAsString", "actualClass"])
for i in range(300):
    prediction = model.predict(X_sample.iloc[i:i+1,:],verbose=0)
    class_labels = np.argmax(prediction[0], axis=0)
    Ypred.at[i, 'sampleId']  = i
    Ypred.at[i, 'classAsIndex']  = class_labels
    Ypred.at[i, 'classAsString']  = enc.get_feature_names(['name'])[class_labels]
    Ypred.at[i, 'actualClass']  = Y_sample.iloc[i].values
    #print(class_labels, enc.get_feature_names(['name'])[class_labels]," Names of class :: ", Y_sample.iloc[i].values)

from IPython.display import display, HTML
display(HTML(Ypred.to_html()))

,sampleId,classAsIndex,classAsString,actualClass
0,0,116,name_German_Khan,[German_Khan]
1,1,116,name_German_Khan,[German_Khan]
2,2,116,name_German_Khan,[German_Khan]
3,3,116,name_German_Khan,[German_Khan]
4,4,116,name_German_Khan,[German_Khan]
5,5,116,name_German_Khan,[German_Khan]
6,6,116,name_German_Khan,[German_Khan]
7,7,116,name_German_Khan,[German_Khan]
8,8,116,name_German_Khan,[German_Khan]
9,9,116,name_German_Khan,[German_Khan]
